In [ ]:
import tensorflow as tf
from tqdm import tqdm_notebook

In [ ]:
import numpy as np
from keras.datasets import cifar10
from sklearn.model_selection import train_test_split
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=10**4, random_state=42)

class_names = np.array(['airplane','automobile ','bird ','cat ','deer ','dog ','frog ','horse ','ship ','truck'])

print (X_train.shape,y_train.shape)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=[12,10])
for i in range(12):
    plt.subplot(3, 4, i + 1)
    plt.xlabel(class_names[y_train[i, 0]])
    plt.imshow(X_train[i])

In [ ]:
X_train = X_train / 255
X_val = X_val / 255
X_test = X_test / 255

In [ ]:
assert np.shape(X_train) == (40000, 32, 32, 3), "data shape should not change"
assert 0.9 <= max(map(np.max, (X_train, X_val, X_test))) <= 1.05
assert 0.0 <= min(map(np.min, (X_train, X_val, X_test))) <= 0.1
assert len(np.unique(X_test / 255.)) > 10, "make sure you casted data to float type"

In [ ]:
import keras
from keras import layers as L
from keras import backend as K

In [ ]:
model = keras.models.Sequential([
    keras.Input(shape=(32, 32, 3)),
    keras.layers.Flatten(),
    #первый полносвязный слой
    keras.layers.Dense(32, activation="tanh"),
    #второй полносвяный слой
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(10, activation="softmax"),]
)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
y_train_labels = tf.keras.utils.to_categorical(y_train)
model.fit(X_train, 
          y_train_labels,
          batch_size=32, 
          epochs=5,
          validation_split=0.2)

In [ ]:
dummy_pred = model.predict(X_train[:20])
assert dummy_pred.shape == (20, 10)
assert np.allclose(dummy_pred.sum(-1), 1)
print("Успех!")


# Обучение сети
Задание 1.1 (обязательно) Будем минимизировать многоклассовую кроссэкнропию с помощью sgd. Вам нужно получить сеть, которая достигнет не менее 45% accuracy на тестовых данных.

In [ ]:
from tensorflow.python.keras.callbacks import TensorBoard
from time import time

In [ ]:
model = keras.models.Sequential([
    keras.Input(shape=(32, 32, 3)),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dense(10, activation="softmax"),]
)
model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
tensorboard = TensorBoard(log_dir = "logs/{}".format(time()))
tensorboard_cbk = keras.callbacks.TensorBoard(log_dir='sample/', histogram_freq=1, write_graph=True, write_images=True)
model.fit(X_train, 
          y_train_labels,
          batch_size=32 , 
          epochs=30,
          callbacks=[tensorboard_cbk],
          validation_split=0.2) 
from sklearn.metrics import accuracy_score
print ("test accuracy: ", accuracy_score(y_test, np.argmax(model.predict(X_test),axis=1)))

In [ ]:
keras.callbacks.History


In [ ]:
from sklearn.metrics import accuracy_score
test_acc = accuracy_score(y_test, np.argmax(model.predict(X_test),axis=1))
print("\n Test_acc =", test_acc)
assert test_acc > 0.45, "Not good enough. Back to the drawing board :)"
print(" Not bad!")

# Карманная сверточная сеть
Задание 1.2 (обязательно) Реализуйте небольшую свёрточную сеть. Совсем небольшую:

In [ ]:
model = keras.models.Sequential([
    keras.Input(shape=(32, 32, 3)),
    keras.layers.Conv2D(filters=10,
                           kernel_size=(3, 3),
                           padding='same',
                           activation='relu',
                           input_shape=X_train.shape[1:]),
    keras.layers.MaxPool2D(pool_size=(2, 2), padding='valid'),
    keras.layers.Flatten(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax"),]
)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
y_train_labels = tf.keras.utils.to_categorical(y_train)
model.fit(X_train, 
          y_train_labels,
          batch_size=32, 
          epochs=20,
          validation_split=0.2) 
print('Test Accuracy: ', accuracy_score(y_test, np.argmax(model.predict(X_test),axis=1)))

In [ ]:
from sklearn.metrics import accuracy_score
test_acc = accuracy_score(y_test, np.argmax(model.predict(X_test),axis=1))
print("\n Test_acc =", test_acc)
assert test_acc > 0.50, "Not good enough. Back to the drawing board :)"
print(" Not bad!")